In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings("ignore")

In [2]:
names_df = pd.read_csv('./Seoul.csv', index_col=0)
names_df

,0
0,강남구 대치동 지지미
1,강남구 대치동 압구정 찹쌀순대 본점
2,강남구 대치동 대찬횟집
3,강남구 대치동 영카츠(YOUNGKATSU) 대치본점
4,강남구 대치동 정통일식 이상마루 돈까츠
...,...
98,강남구 대치동 최가네버섯매운탕칼국수
99,강남구 대치동 휘초밥
100,강남구 대치동 리즈소바
101,강남구 대치동 스시아오마츠


In [3]:
options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver'

source_url = "https://map.kakao.com/"

page_urls = []

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for name in names_df['0']:
    try:
        driver.get(source_url)

        search_bar = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
        search_bar.send_keys(name)

        searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
        driver.execute_script("arguments[0].click();", searchbutton)

        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="info.search.place.list"]/li[2]/div[3]/strong/a[1]')))

        html = driver.page_source

        soup = BeautifulSoup(html, "html.parser")
        moreview = soup.find(name="a", attrs={"class":"moreview"})
    
        page_url = moreview.get("href")
        print(page_url)
        page_urls.append(page_url)
    except:
        pass

driver.close()

https://place.map.kakao.com/8123533
https://place.map.kakao.com/311269730
https://place.map.kakao.com/1488731354
https://place.map.kakao.com/1234400636
https://place.map.kakao.com/489689419
https://place.map.kakao.com/16765944
https://place.map.kakao.com/977331811
https://place.map.kakao.com/1230712749
https://place.map.kakao.com/10042991
https://place.map.kakao.com/652582950
https://place.map.kakao.com/17606011
https://place.map.kakao.com/10030025
https://place.map.kakao.com/1440229255
https://place.map.kakao.com/199819878
https://place.map.kakao.com/920529477
https://place.map.kakao.com/17321255
https://place.map.kakao.com/293125397
https://place.map.kakao.com/11502832
https://place.map.kakao.com/1544726790
https://place.map.kakao.com/8148073
https://place.map.kakao.com/751059202
https://place.map.kakao.com/22873109
https://place.map.kakao.com/1647819710
https://place.map.kakao.com/2113270821


In [12]:
columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for page_url in page_urls:
    try:
        driver.get(page_url)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2')))
        

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        names = soup.find(name='h2', attrs={"class":"tit_location"})
        
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
    
    
        for rate, review in zip(rates, reviews):
            row = [names.text, rate.text[0], review.find(name="span").text]
            series = pd.Series(row, index=review_df.columns)
            review_df = review_df.append(series, ignore_index=True)

        for button_num in range(2, 6):
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            driver.implicitly_wait(5)
            
        
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                row = [names.text, rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=review_df.columns)
                review_df = review_df.append(series, ignore_index=True)
    except:
        pass

    
driver.close()

In [10]:
review_df

,name,score,review
0,피양콩할마니 본점,1,1인분은 4시까지만!
1,피양콩할마니 본점,5,자극적이지 않고 슴슴하면서 반찬과도 잘 어울리는 콩비지 전골입니다. 할머니도 반갑게...
2,피양콩할마니 본점,3,건강식으로 추천 친절하지는 않음 ㅠ
3,피양콩할마니 본점,4,김치콩비지 부드럽고 깊은 콩맛이 느껴짐. 슴슴하니 좋아요. 반찬은 별로네요.
4,피양콩할마니 본점,5,여기 진짜 맛있는데 ~ 갑자기 먹고싶어서 배달되나 살펴보러 왔다가 평보고 조금 놀랐...
...,...,...,...
178,오레노카츠 선릉점,5,주말에도 점심특선 주문되고 정식세트에 있는 생카츠를 다른 돈까스로 변경됩니다~(저는...
179,오레노카츠 선릉점,1,호천당 시절엔 친절하고 좋았는데 상호 바뀌고 나서 별로입니다. 문 열 때부터 단골인...
180,동경에서 먹었던 규동,2,먹는물 따르는 정수기 바로옆에 더러운 빗자루와 쓰레받기 너무 찝찝해요. 위생적으로나...
181,동경에서 먹었던 규동,5,음식이 조금 짜긴 하지만 가격도 나쁘지 않고 맛있게 먹었어요. 직원들도 친절하고 내...


In [11]:
review_df.to_csv("review_data.csv", index=False)